In [8]:
from openai import OpenAI
import sys
import time
import pandas as pd
import numpy as np


In [ ]:
client = OpenAI(
    api_key="secret"
)

In [ ]:
#spliting the dataset
file_path = r"C:\Users\sdtnt\cse256\Final_Project2\data\Pew_American_Trends_Panel_disagreement_500.csv"
data = pd.read_csv(file_path, on_bad_lines="skip", delimiter="\t")


# shuffle the dataset for randomness
data_shuffled = data.sample(frac=1, random_state=42).reset_index(drop=True)

# split data into non-overlapping subsets
def split_into_subsets(data, num_subsets):
    subsets = []
    subset_size = len(data) // num_subsets
    
    for i in range(num_subsets):
        start_idx = i * subset_size
        end_idx = (i + 1) * subset_size if i != num_subsets - 1 else len(data)
        subsets.append(data.iloc[start_idx:end_idx])
    
    return subsets

num_subsets = 5
subsets = split_into_subsets(data_shuffled, num_subsets)

# Write each subset to a separate file
for idx, subset in enumerate(subsets):
    output_file = f'subset_{idx + 1}.csv'
    subset.to_csv(output_file, index=False)
    print(f"Subset {idx + 1} written to {output_file}")


Subset 1 written to subset_1.csv
Subset 2 written to subset_2.csv
Subset 3 written to subset_3.csv
Subset 4 written to subset_4.csv
Subset 5 written to subset_5.csv


In [35]:
prompt_base = f''' '''

prompt_explain = f'''Explain your answer.'''

prompt_arg = f'''Write a 200 word essay to argue your answer.'''

prompt_pick = f'''Pick an opinion and write a 200 word argumentative essay.'''

prompt_detailed = f'''Pick one perspective to answer this question. Write a 200 word essay to argue only your side of the argument'''

prompt_evidence = f'''Write a 200 word argumentative essay to answer this question include evidence to back your claim.'''
# prompt_lib = f'''Take the standpoint of a liberal. Write a 200 word argumentative essay'''

# prompt_con = f'''Take the standpoint of a conservative. Write a 200 word argumentative essay'''


prompts = (prompt_base,
    prompt_explain,
    prompt_arg,
    prompt_pick,
    prompt_detailed,
    prompt_evidence)

In [ ]:
def get_opinion(question, prompt):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": '\n'.join(question) + ' ' + prompt
            }
        ]
    )
    return completion.choices[0].message.content

In [53]:
def test():
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": "In order to address economic inequality in this country, do you think the government Should raise taxes on people like you', 'Should not raise taxes on people like you' Write a 200 word argumentative essay to answer this question include evidence to back your claim."
            }
        ]
    )

    print(completion.choices[0].message.content)

test()

As an AI assistant, I do not have personal wealth or income, so the question of whether taxes should be raised on people like me is moot. However, I can provide arguments on both sides of the issue.

Advocates for raising taxes on the wealthy argue that it is a way to address economic inequality, as those with higher incomes can afford to contribute more towards public services and social welfare programs. They point to research that shows a correlation between high levels of economic inequality and social problems such as crime rates, health disparities, and lower educational attainment.

On the other hand, opponents of raising taxes on the wealthy argue that it can stifle economic growth and discourage investment, ultimately harming job creation and overall prosperity. They point to the potential for wealthy individuals to seek out tax avoidance strategies or move their assets offshore, reducing tax revenue without addressing the root causes of economic inequality.

In conclusion, th

In [38]:
import csv

In [50]:
with open('baseline_prompt.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)

    for index, row in enumerate(csv_reader):
        question = row.get('question', None)
        filename = f"{index + 1}.txt"
                    
        # Write the generated output to a file
        with open(filename, 'w') as output_file:
            if question:
                for prompt in prompts:
                    header = '\n' + question + prompt + '\n'
                    output = get_opinion(question, prompt)
                    output_file.write(header)
                    output_file.write(output)
                    output_file.write('\n \n ----------------------------------------------------------------')
                

In [51]:
with open('100_Q.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    with open('100_Q_GPT_responses', 'w') as output_file:
        for index, row in enumerate(csv_reader):
            question = row.get('question', None)
            if question:
                header = '\n' + question + prompt + '\n'
                output = get_opinion(question, prompt_evidence)
                output_file.write(header)
                output_file.write(output)
                output_file.write('\n \n ----------------------------------------------------------------')